# 🔄 Internal Feedback Loop System

## JSON-Based Evaluation Pipeline for Easy Language Output

**Model**: `llama-3.1-8b-instant` (Groq)

---

### Purpose

This notebook implements a structured feedback loop system using JSON files to:
1. Store and track model outputs
2. Evaluate outputs against Easy Language rules
3. Generate actionable feedback for prompt/guardrail improvements
4. Manage development tickets for systematic improvements

### Architecture

```
┌─────────────────────────────────────────────────────────────────────┐
│                    JSON-BASED FEEDBACK LOOP                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  inputs/           outputs/           feedback/          tickets/   │
│  ────────          ─────────          ──────────         ─────────  │
│  source_texts.json → model_outputs.json → eval_results.json → tasks.json │
│       │                  │                    │                │     │
│       ▼                  ▼                    ▼                ▼     │
│  [Original]        [Simplified]         [Scores +        [Dev Tasks] │
│   Texts              Texts              Violations]                  │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

# 📋 Development Ticket Structure

## Sprint Backlog: Internal Feedback Loop System

| Ticket ID | Title | Priority | Status | Estimate |
|-----------|-------|----------|--------|----------|
| FEED-001 | Define JSON schemas for inputs/outputs | High | ⬜ Todo | 2h |
| FEED-002 | Implement scoring functions | High | ⬜ Todo | 4h |
| FEED-003 | Create verification test suite | High | ⬜ Todo | 3h |
| FEED-004 | Build feedback aggregation | Medium | ⬜ Todo | 3h |
| FEED-005 | Implement ticket generation from findings | Medium | ⬜ Todo | 2h |
| FEED-006 | Create evaluation dashboard view | Low | ⬜ Todo | 4h |
| FEED-007 | Add historical tracking | Low | ⬜ Todo | 2h |

In [1]:
# Setup: Import required packages
import json
import os
import re
from datetime import datetime
from typing import Dict, List, Optional
from dataclasses import dataclass, asdict
from enum import Enum
import pandas as pd
import numpy as np

# Create feedback loop directories
FEEDBACK_DIR = "../outputs/feedback_loop"
os.makedirs(f"{FEEDBACK_DIR}/inputs", exist_ok=True)
os.makedirs(f"{FEEDBACK_DIR}/outputs", exist_ok=True)
os.makedirs(f"{FEEDBACK_DIR}/evaluations", exist_ok=True)
os.makedirs(f"{FEEDBACK_DIR}/tickets", exist_ok=True)

print("✅ Feedback loop directories created")

✅ Feedback loop directories created


---

# 🎫 FEED-001: JSON Schema Definitions

**Status**: ⬜ Todo → 🔄 In Progress

**Acceptance Criteria**:
- [x] Define schema for source texts
- [x] Define schema for model outputs
- [x] Define schema for evaluation results
- [x] Define schema for development tickets

In [2]:
# FEED-001: JSON Schema Definitions

class TicketStatus(str, Enum):
    TODO = "todo"
    IN_PROGRESS = "in_progress"
    REVIEW = "review"
    DONE = "done"
    BLOCKED = "blocked"

class Priority(str, Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"

@dataclass
class SourceText:
    """Schema for input source texts."""
    id: str
    text: str
    language: str  # "de" or "en"
    domain: str    # e.g., "legal", "medical", "government"
    difficulty: str  # "standard", "complex", "technical"
    created_at: str = None
    
    def __post_init__(self):
        if self.created_at is None:
            self.created_at = datetime.now().isoformat()

@dataclass
class ModelOutput:
    """Schema for model-generated simplified texts."""
    id: str
    source_id: str
    model: str
    prompt_version: str
    output_text: str
    language: str
    inference_time_ms: float
    timestamp: str = None
    metadata: Dict = None
    
    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = datetime.now().isoformat()
        if self.metadata is None:
            self.metadata = {}

@dataclass
class EvaluationResult:
    """Schema for evaluation results."""
    id: str
    output_id: str
    source_id: str
    
    # Readability metrics
    ari_score: float
    lix_score: float
    avg_sentence_len: float
    max_sentence_len: int
    pct_long_sentences: float
    
    # Semantic metrics
    meaning_cosine: float
    
    # Rule compliance
    rule_violations: List[str]
    rule_score: float  # 0-1 scale
    
    # Overall
    passed_guardrails: bool
    overall_score: float
    
    timestamp: str = None
    
    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = datetime.now().isoformat()

@dataclass
class DevelopmentTicket:
    """Schema for development tickets."""
    id: str
    title: str
    description: str
    ticket_type: str  # "bug", "improvement", "feature", "investigation"
    priority: str
    status: str
    related_evaluations: List[str]  # List of evaluation IDs
    suggested_action: str
    acceptance_criteria: List[str]
    created_at: str = None
    updated_at: str = None
    
    def __post_init__(self):
        if self.created_at is None:
            self.created_at = datetime.now().isoformat()
        if self.updated_at is None:
            self.updated_at = self.created_at

print("✅ FEED-001: JSON schemas defined")

✅ FEED-001: JSON schemas defined


---

# 🎫 FEED-002: Scoring Functions

**Status**: ⬜ Todo → 🔄 In Progress

**Acceptance Criteria**:
- [x] Implement ARI score calculation
- [x] Implement LIX score calculation
- [x] Implement sentence length metrics
- [x] Implement meaning similarity
- [x] Implement rule violation detection

In [3]:
# FEED-002: Scoring Functions

def compute_ari(text: str) -> float:
    """Compute Automated Readability Index (works for EN + DE)."""
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    words = re.findall(r'\b\w+\b', text)
    if not sentences or not words:
        return 0.0
    chars = sum(len(w) for w in words)
    return 4.71 * (chars / len(words)) + 0.5 * (len(words) / len(sentences)) - 21.43

def compute_lix(text: str) -> float:
    """Compute LIX readability index."""
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    words = re.findall(r'\b\w+\b', text)
    if not sentences or not words:
        return 0.0
    long_words = [w for w in words if len(w) > 6]
    return (len(words) / len(sentences)) + (len(long_words) * 100 / len(words))

def compute_sentence_metrics(text: str) -> Dict:
    """Compute sentence-level metrics."""
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    if not sentences:
        return {"avg": 0, "max": 0, "pct_long": 0}
    
    sentence_lens = [len(re.findall(r'\b\w+\b', s)) for s in sentences]
    return {
        "avg": np.mean(sentence_lens),
        "max": max(sentence_lens),
        "pct_long": sum(1 for l in sentence_lens if l > 15) / len(sentence_lens) * 100
    }

def compute_meaning_similarity(source: str, output: str) -> float:
    """Compute cosine similarity using sentence-transformers."""
    try:
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        embeddings = model.encode([source, output])
        sim = np.dot(embeddings[0], embeddings[1]) / (
            np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1])
        )
        return float(sim)
    except ImportError:
        print("Warning: sentence-transformers not installed, using TF-IDF fallback")
        return compute_tfidf_similarity(source, output)

def compute_tfidf_similarity(source: str, output: str) -> float:
    """Fallback TF-IDF similarity."""
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    try:
        vectorizer = TfidfVectorizer()
        tfidf = vectorizer.fit_transform([source, output])
        return float(cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0])
    except:
        return 0.0

print("✅ FEED-002: Scoring functions implemented")

✅ FEED-002: Scoring functions implemented


In [4]:
# FEED-002: Rule Violation Detection

# Easy Language rules based on prompt templates
EASY_LANGUAGE_RULES = {
    "de": {
        "max_sentence_words": 10,
        "use_sie_or_du": True,
        "avoid_passive": True,
        "avoid_negations": True,
        "use_bullets_for_lists": True,
        "explain_technical_terms": True,
        "consistent_terminology": True,
    },
    "en": {
        "max_sentence_words": 10,
        "use_you": True,
        "avoid_passive": True,
        "avoid_negations": True,
        "use_bullets_for_lists": True,
        "explain_technical_terms": True,
        "consistent_terminology": True,
    }
}

def detect_rule_violations(text: str, language: str = "de") -> List[str]:
    """Detect Easy Language rule violations."""
    violations = []
    rules = EASY_LANGUAGE_RULES.get(language, EASY_LANGUAGE_RULES["de"])
    
    # Check sentence length
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    for i, s in enumerate(sentences):
        word_count = len(re.findall(r'\b\w+\b', s))
        if word_count > rules["max_sentence_words"]:
            violations.append(f"sentence_{i+1}_too_long: {word_count} words (max {rules['max_sentence_words']})")
    
    # Check for passive voice patterns (German)
    if language == "de" and rules["avoid_passive"]:
        passive_patterns = [r'\bwird\b.*\b(ge\w+t|ge\w+en)\b', r'\bwurde\b', r'\bworden\b']
        for pattern in passive_patterns:
            if re.search(pattern, text, re.IGNORECASE):
                violations.append("passive_voice_detected")
                break
    
    # Check for passive voice patterns (English)
    if language == "en" and rules["avoid_passive"]:
        passive_patterns = [r'\bwas\s+\w+ed\b', r'\bwere\s+\w+ed\b', r'\bis\s+being\b', r'\bhas\s+been\b']
        for pattern in passive_patterns:
            if re.search(pattern, text, re.IGNORECASE):
                violations.append("passive_voice_detected")
                break
    
    # Check for negations
    if rules["avoid_negations"]:
        negation_patterns_de = [r'\bnicht\b', r'\bkein\w*\b', r'\bnie\b', r'\bniemals\b']
        negation_patterns_en = [r"\bdon't\b", r"\bdoesn't\b", r"\bwon't\b", r"\bcan't\b", 
                                r"\bnot\b", r"\bnever\b", r"\bno\b"]
        patterns = negation_patterns_de if language == "de" else negation_patterns_en
        for pattern in patterns:
            if re.search(pattern, text, re.IGNORECASE):
                violations.append(f"negation_detected: {pattern}")
    
    # Check for intro/outro text
    intro_patterns = [
        r'^(Hier ist|Here is)',
        r'^(Der vereinfachte Text|The simplified text)',
        r'^(Einfache Version|Simple version)',
    ]
    for pattern in intro_patterns:
        if re.search(pattern, text, re.IGNORECASE):
            violations.append("contains_intro_text")
            break
    
    # Check for XML/HTML tags
    if re.search(r'<[^>]+>', text):
        violations.append("contains_xml_tags")
    
    return violations

def compute_rule_score(violations: List[str], total_sentences: int) -> float:
    """Compute rule compliance score (1.0 = perfect, 0.0 = all rules violated)."""
    if total_sentences == 0:
        return 0.0
    
    # Weight different violations
    weights = {
        "sentence_too_long": 0.15,
        "passive_voice": 0.1,
        "negation": 0.1,
        "intro_text": 0.2,
        "xml_tags": 0.3,
    }
    
    penalty = 0.0
    for v in violations:
        for key, weight in weights.items():
            if key in v.lower():
                penalty += weight
                break
        else:
            penalty += 0.05  # Default penalty for unknown violations
    
    return max(0.0, 1.0 - penalty)

print("✅ FEED-002: Rule violation detection implemented")

✅ FEED-002: Rule violation detection implemented


---

# 🎫 FEED-003: Verification Test Suite

**Status**: ⬜ Todo → 🔄 In Progress

**Acceptance Criteria**:
- [x] Create test cases from prompt template examples
- [x] Implement verification functions
- [x] Generate test reports

In [5]:
# FEED-003: Verification Test Suite

# Test cases derived from prompt template examples
VERIFICATION_TESTS = {
    "de": [
        {
            "id": "de_test_001",
            "source": "Der Antragsteller muss die erforderlichen Unterlagen innerhalb der gesetzlich vorgeschriebenen Frist beim zuständigen Amt einreichen.",
            "expected_output": """Sie müssen Papiere abgeben.

* Bringen Sie die Papiere zum Amt.
* Sie haben dafür eine bestimmte Zeit.
* Diese Zeit steht im Gesetz.""",
            "domain": "government",
            "expected_rules": ["uses_bullets", "short_sentences", "direct_address"]
        },
        {
            "id": "de_test_002",
            "source": "Die Medikamenteneinnahme sollte zweimal täglich zu den Mahlzeiten erfolgen, um mögliche Magen-Darm-Beschwerden zu minimieren.",
            "expected_output": """Nehmen Sie diese Medizin zwei Mal am Tag.

* Nehmen Sie sie beim Essen.
* Das ist besser für Ihren Magen.""",
            "domain": "medical",
            "expected_rules": ["uses_bullets", "short_sentences", "direct_address"]
        }
    ],
    "en": [
        {
            "id": "en_test_001",
            "source": "Upon arrival at the facility, visitors are required to sign in at the front desk and present valid photo identification.",
            "expected_output": """When you arrive:

* Go to the front desk.
* Sign your name.
* Show your photo ID.""",
            "domain": "general",
            "expected_rules": ["uses_bullets", "short_sentences", "direct_address"]
        },
        {
            "id": "en_test_002",
            "source": "The medication should be administered twice daily with food to minimize potential gastrointestinal discomfort.",
            "expected_output": """Take this medicine two times every day.

* Eat food when you take it.
* This helps your stomach feel better.""",
            "domain": "medical",
            "expected_rules": ["uses_bullets", "short_sentences", "direct_address"]
        }
    ]
}

print(f"✅ Loaded {len(VERIFICATION_TESTS['de'])} German test cases")
print(f"✅ Loaded {len(VERIFICATION_TESTS['en'])} English test cases")

✅ Loaded 2 German test cases
✅ Loaded 2 English test cases


In [6]:
# FEED-003: Verification Functions

def verify_output_quality(source: str, output: str, expected: str, language: str) -> Dict:
    """Verify output quality against expected results."""
    
    # Compute metrics for actual output
    sentence_metrics = compute_sentence_metrics(output)
    
    verification = {
        "ari_score": round(compute_ari(output), 2),
        "lix_score": round(compute_lix(output), 2),
        "avg_sentence_len": round(sentence_metrics["avg"], 1),
        "max_sentence_len": sentence_metrics["max"],
        "pct_long_sentences": round(sentence_metrics["pct_long"], 1),
        "meaning_similarity": round(compute_meaning_similarity(source, output), 3),
        "rule_violations": detect_rule_violations(output, language),
        "uses_bullets": "*" in output or "•" in output or "-" in output,
        "has_paragraphs": "\n\n" in output or output.count("\n") > 1,
    }
    
    # Compute similarity to expected output
    verification["expected_similarity"] = round(
        compute_meaning_similarity(expected, output), 3
    )
    
    # Check guardrails
    guardrails = {
        "ari_pass": verification["ari_score"] <= 8,
        "lix_pass": verification["lix_score"] <= 40,
        "sentence_len_pass": verification["avg_sentence_len"] <= 15,
        "meaning_pass": verification["meaning_similarity"] >= 0.70,
        "rules_pass": len(verification["rule_violations"]) == 0,
    }
    verification["guardrails"] = guardrails
    verification["all_guardrails_pass"] = all(guardrails.values())
    
    return verification

def run_verification_suite(test_cases: List[Dict], model_outputs: Dict[str, str]) -> pd.DataFrame:
    """Run verification suite on model outputs."""
    results = []
    
    for test in test_cases:
        test_id = test["id"]
        if test_id not in model_outputs:
            results.append({
                "test_id": test_id,
                "status": "SKIPPED",
                "reason": "No model output found"
            })
            continue
        
        output = model_outputs[test_id]
        language = "de" if test_id.startswith("de_") else "en"
        
        verification = verify_output_quality(
            source=test["source"],
            output=output,
            expected=test["expected_output"],
            language=language
        )
        
        results.append({
            "test_id": test_id,
            "domain": test["domain"],
            "status": "PASS" if verification["all_guardrails_pass"] else "FAIL",
            **verification
        })
    
    return pd.DataFrame(results)

print("✅ FEED-003: Verification functions implemented")

✅ FEED-003: Verification functions implemented


---

# 🎫 FEED-004: Feedback Aggregation

**Status**: ⬜ Todo → 🔄 In Progress

**Acceptance Criteria**:
- [x] Aggregate evaluation results
- [x] Identify patterns in failures
- [x] Generate actionable recommendations

In [7]:
# FEED-004: Feedback Aggregation

def aggregate_feedback(evaluations: List[Dict]) -> Dict:
    """Aggregate evaluation results into feedback summary."""
    
    df = pd.DataFrame(evaluations)
    
    # Overall statistics
    summary = {
        "total_evaluations": len(df),
        "pass_rate": (df["all_guardrails_pass"].sum() / len(df) * 100) if len(df) > 0 else 0,
        "metrics": {
            "avg_ari": df["ari_score"].mean(),
            "avg_lix": df["lix_score"].mean(),
            "avg_sentence_len": df["avg_sentence_len"].mean(),
            "avg_meaning_similarity": df["meaning_similarity"].mean(),
        },
        "guardrail_pass_rates": {},
        "common_violations": {},
        "recommendations": [],
    }
    
    # Guardrail pass rates
    if "guardrails" in df.columns:
        guardrail_cols = df["guardrails"].apply(pd.Series)
        for col in guardrail_cols.columns:
            summary["guardrail_pass_rates"][col] = guardrail_cols[col].mean() * 100
    
    # Common violations
    all_violations = []
    for violations in df["rule_violations"]:
        if isinstance(violations, list):
            all_violations.extend(violations)
    
    from collections import Counter
    violation_counts = Counter(all_violations)
    summary["common_violations"] = dict(violation_counts.most_common(10))
    
    # Generate recommendations
    recommendations = []
    
    if summary["metrics"]["avg_ari"] > 8:
        recommendations.append({
            "type": "prompt",
            "issue": "High ARI score",
            "action": "Add instruction to use simpler, shorter words",
            "priority": "high"
        })
    
    if summary["metrics"]["avg_sentence_len"] > 10:
        recommendations.append({
            "type": "prompt",
            "issue": "Sentences too long",
            "action": "Emphasize max 10 words per sentence rule",
            "priority": "high"
        })
    
    if summary["metrics"]["avg_meaning_similarity"] < 0.70:
        recommendations.append({
            "type": "prompt",
            "issue": "Meaning not preserved",
            "action": "Add instruction to keep all key information",
            "priority": "critical"
        })
    
    if "passive_voice_detected" in summary["common_violations"]:
        recommendations.append({
            "type": "prompt",
            "issue": "Passive voice usage",
            "action": "Add explicit examples of active voice transformation",
            "priority": "medium"
        })
    
    summary["recommendations"] = recommendations
    
    return summary

def generate_feedback_report(summary: Dict) -> str:
    """Generate human-readable feedback report."""
    lines = [
        "# 📊 Feedback Loop Report",
        f"\n**Generated**: {datetime.now().strftime('%Y-%m-%d %H:%M')}",
        f"\n## Summary",
        f"- Total Evaluations: {summary['total_evaluations']}",
        f"- Pass Rate: {summary['pass_rate']:.1f}%",
        f"\n## Metrics",
        f"| Metric | Value | Target | Status |",
        f"|--------|-------|--------|--------|",
        f"| ARI Score | {summary['metrics']['avg_ari']:.1f} | ≤ 8 | {'✅' if summary['metrics']['avg_ari'] <= 8 else '❌'} |",
        f"| LIX Score | {summary['metrics']['avg_lix']:.1f} | ≤ 40 | {'✅' if summary['metrics']['avg_lix'] <= 40 else '❌'} |",
        f"| Avg Sentence Len | {summary['metrics']['avg_sentence_len']:.1f} | ≤ 10 | {'✅' if summary['metrics']['avg_sentence_len'] <= 10 else '⚠️'} |",
        f"| Meaning Similarity | {summary['metrics']['avg_meaning_similarity']:.2f} | ≥ 0.70 | {'✅' if summary['metrics']['avg_meaning_similarity'] >= 0.70 else '❌'} |",
    ]
    
    if summary["common_violations"]:
        lines.append("\n## Common Violations")
        for violation, count in summary["common_violations"].items():
            lines.append(f"- {violation}: {count}x")
    
    if summary["recommendations"]:
        lines.append("\n## 💡 Recommendations")
        for rec in summary["recommendations"]:
            priority_icon = {"critical": "🔴", "high": "🟠", "medium": "🟡", "low": "🟢"}
            lines.append(f"\n### {priority_icon.get(rec['priority'], '⚪')} {rec['issue']}")
            lines.append(f"- **Type**: {rec['type']}")
            lines.append(f"- **Action**: {rec['action']}")
    
    return "\n".join(lines)

print("✅ FEED-004: Feedback aggregation implemented")

✅ FEED-004: Feedback aggregation implemented


---

# 🎫 FEED-005: Ticket Generation from Findings

**Status**: ⬜ Todo → 🔄 In Progress

**Acceptance Criteria**:
- [x] Auto-generate tickets from evaluation findings
- [x] Prioritize tickets based on impact
- [x] Save tickets to JSON

In [8]:
# FEED-005: Ticket Generation

def generate_tickets_from_feedback(summary: Dict, evaluation_ids: List[str]) -> List[DevelopmentTicket]:
    """Generate development tickets from feedback summary."""
    tickets = []
    ticket_counter = 1
    
    for rec in summary["recommendations"]:
        ticket_type = "improvement" if rec["type"] == "prompt" else "bug"
        
        ticket = DevelopmentTicket(
            id=f"AUTO-{datetime.now().strftime('%Y%m%d')}-{ticket_counter:03d}",
            title=f"[{rec['type'].upper()}] {rec['issue']}",
            description=f"Automated ticket generated from feedback loop analysis.\n\n"
                        f"**Issue**: {rec['issue']}\n"
                        f"**Suggested Action**: {rec['action']}",
            ticket_type=ticket_type,
            priority=rec["priority"],
            status="todo",
            related_evaluations=evaluation_ids[:5],  # Link to first 5 related evaluations
            suggested_action=rec["action"],
            acceptance_criteria=[
                f"Pass rate improves after implementation",
                f"{rec['issue']} occurrences reduced by >50%",
                "All verification tests pass"
            ]
        )
        tickets.append(ticket)
        ticket_counter += 1
    
    # Add violation-specific tickets
    for violation, count in summary["common_violations"].items():
        if count >= 3:  # Only create tickets for recurring violations
            ticket = DevelopmentTicket(
                id=f"AUTO-{datetime.now().strftime('%Y%m%d')}-{ticket_counter:03d}",
                title=f"[VIOLATION] Fix recurring: {violation.split(':')[0]}",
                description=f"This violation occurred {count} times in the evaluation batch.\n\n"
                            f"**Violation**: {violation}\n"
                            f"**Occurrences**: {count}",
                ticket_type="bug",
                priority="high" if count >= 5 else "medium",
                status="todo",
                related_evaluations=evaluation_ids[:3],
                suggested_action=f"Add explicit rule to prevent {violation.split(':')[0]}",
                acceptance_criteria=[
                    f"{violation.split(':')[0]} occurrences drop to 0",
                    "No regression in other metrics"
                ]
            )
            tickets.append(ticket)
            ticket_counter += 1
    
    return tickets

def save_tickets_to_json(tickets: List[DevelopmentTicket], filepath: str):
    """Save tickets to JSON file."""
    ticket_data = {
        "generated_at": datetime.now().isoformat(),
        "total_tickets": len(tickets),
        "tickets": [asdict(t) for t in tickets]
    }
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(ticket_data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Saved {len(tickets)} tickets to {filepath}")

print("✅ FEED-005: Ticket generation implemented")

✅ FEED-005: Ticket generation implemented


---

# 🚀 Complete Feedback Loop Pipeline

## Integration with llama-3.1-8b-instant

In [9]:
# Model Configuration for llama-3.1-8b-instant

MODEL_CONFIG = {
    "model": "llama-3.1-8b-instant",
    "provider": "groq",
    "temperature": 0.3,
    "max_tokens": 1024,
    "prompt_version": "v1.0",
}

# Prompt templates
SYSTEM_PROMPT_DE = """# Identität

Du bist ein Experte für Einfache Sprache. Du machst komplexe Informationen 
für alle Menschen zugänglich, auch für Menschen mit Lernschwierigkeiten oder geringer Lesekompetenz.

# Anweisungen

*   Schreibe den Text in sehr einfacher, leicht verständlicher Sprache um (Stufe I).
*   Verwende sehr kurze Sätze (maximal 10 Wörter pro Satz).
*   Verwende nur einfache, alltägliche Wörter. Erkläre ungewöhnliche Wörter in Klammern.
*   Füge Leerzeilen zwischen jeden Absatz ein.
*   Verwende Aufzählungspunkte für Schritte, Listen oder mehrere Punkte. Sonst kurze Sätze.
*   Schreibe KEINE Einleitung oder Schlussworte (z.B. "Hier ist der vereinfachte Text").
*   Gib NUR den vereinfachten Text aus.

# Regeln für Einfache Sprache

* Sprich den Leser direkt mit "Sie" oder "du" an. Verwende einen freundlichen, neutralen Ton.
* Vermeide bürokratische, juristische oder befehlende Sprache.
* Bevorzuge aktive Sprache. Vermeide Passiv wann immer möglich.
* Formuliere positiv. Vermeide Verneinungen und niemals doppelte Verneinungen.
* Verwende einfache, bekannte Wörter. Vermeide Fachbegriffe, Fremdwörter oder formelle Ausdrücke.
* Ersetze abstrakte Substantive durch konkrete, aktive Verben.
* Erkläre notwendige Fachbegriffe oder Abkürzungen beim ersten Auftreten.
* Entferne Füllwörter und unnötige Details. Behalte nur wesentliche Informationen.
* Verwende die gleichen Wörter durchgehend. Wechsle nicht die Begriffe für dieselbe Sache.
* Teile lange Sätze auf. Kein Satz länger als 10 Wörter.
* Halte Subjekt und Verb nah beieinander.
* Verwende eine klare Struktur. Nutze Aufzählungspunkte für Listen oder Schritte."""

SYSTEM_PROMPT_EN = """# Identity

You are an expert in plain language writing specialized in making complex information 
accessible to everyone, including people with learning disabilities or low literacy.

# Instructions

*   Rewrite the input text to be extremely simple and easy to understand (Level I).
*   Use very short sentences (maximum 10 words per sentence).
*   Use only simple, everyday words. Explain any uncommon words in parentheses locally.
*   Add blank lines between every paragraph.
*   Use bullet points for steps, lists, or multiple items. Otherwise use short sentences.
*   Do NOT include any introductory or concluding text (e.g., "Here is the simplified text").
*   Output ONLY the simplified text.

# Plain Language Rules

* Address the reader directly using "you". Use a friendly, neutral tone.
* Avoid bureaucratic, legalistic, or commanding language.
* Prefer active voice. Avoid passive voice whenever possible.
* Use positive wording. Avoid negations and never use double negatives.
* Use simple, familiar words. Avoid technical, foreign, or formal terms.
* Replace abstract nouns with concrete, active verbs.
* Explain necessary technical terms or abbreviations the first time they appear.
* Remove filler words and unnecessary details. Keep only essential information.
* Use the same words consistently. Do not switch terms for the same thing.
* Break up long sentences. No sentence longer than 10 words.
* Keep subjects and verbs close together.
* Use clear structure. Use bullet points for lists or steps."""

print("✅ Model configuration loaded for llama-3.1-8b-instant")

✅ Model configuration loaded for llama-3.1-8b-instant


In [10]:
# Complete Pipeline Class

class FeedbackLoopPipeline:
    """Complete feedback loop pipeline for Easy Language evaluation."""
    
    def __init__(self, model_config: Dict, output_dir: str = "../outputs/feedback_loop"):
        self.model_config = model_config
        self.output_dir = output_dir
        self.run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Create run directory
        self.run_dir = f"{output_dir}/runs/{self.run_id}"
        os.makedirs(self.run_dir, exist_ok=True)
        
        self.inputs = []
        self.outputs = []
        self.evaluations = []
        self.tickets = []
    
    def add_source_texts(self, texts: List[SourceText]):
        """Add source texts to the pipeline."""
        self.inputs.extend(texts)
        print(f"Added {len(texts)} source texts")
    
    def add_model_outputs(self, outputs: List[ModelOutput]):
        """Add model outputs to the pipeline."""
        self.outputs.extend(outputs)
        print(f"Added {len(outputs)} model outputs")
    
    def run_evaluation(self):
        """Run evaluation on all outputs."""
        print(f"\n{'='*60}")
        print(f"Running evaluation for {len(self.outputs)} outputs...")
        print(f"{'='*60}")
        
        for output in self.outputs:
            # Find corresponding source
            source = next((s for s in self.inputs if s.id == output.source_id), None)
            if not source:
                print(f"⚠️ No source found for output {output.id}")
                continue
            
            # Compute metrics
            sentence_metrics = compute_sentence_metrics(output.output_text)
            violations = detect_rule_violations(output.output_text, output.language)
            
            eval_result = EvaluationResult(
                id=f"eval_{output.id}",
                output_id=output.id,
                source_id=source.id,
                ari_score=round(compute_ari(output.output_text), 2),
                lix_score=round(compute_lix(output.output_text), 2),
                avg_sentence_len=round(sentence_metrics["avg"], 1),
                max_sentence_len=sentence_metrics["max"],
                pct_long_sentences=round(sentence_metrics["pct_long"], 1),
                meaning_cosine=round(compute_meaning_similarity(source.text, output.output_text), 3),
                rule_violations=violations,
                rule_score=round(compute_rule_score(violations, len(sentence_metrics)), 2),
                passed_guardrails=False,  # Will be set below
                overall_score=0.0,  # Will be computed
            )
            
            # Check guardrails
            eval_result.passed_guardrails = (
                eval_result.ari_score <= 8 and
                eval_result.lix_score <= 40 and
                eval_result.avg_sentence_len <= 15 and
                eval_result.meaning_cosine >= 0.70 and
                len(eval_result.rule_violations) == 0
            )
            
            # Compute overall score (weighted average)
            eval_result.overall_score = round(
                0.3 * (1 - min(eval_result.ari_score / 15, 1)) +
                0.2 * (1 - min(eval_result.avg_sentence_len / 20, 1)) +
                0.3 * eval_result.meaning_cosine +
                0.2 * eval_result.rule_score,
                3
            )
            
            self.evaluations.append(eval_result)
        
        passed = sum(1 for e in self.evaluations if e.passed_guardrails)
        print(f"\n✅ Evaluation complete: {passed}/{len(self.evaluations)} passed guardrails")
    
    def generate_feedback(self) -> Dict:
        """Generate feedback summary."""
        # Convert evaluations to dicts for aggregation
        eval_dicts = []
        for e in self.evaluations:
            eval_dicts.append({
                "ari_score": e.ari_score,
                "lix_score": e.lix_score,
                "avg_sentence_len": e.avg_sentence_len,
                "meaning_similarity": e.meaning_cosine,
                "rule_violations": e.rule_violations,
                "all_guardrails_pass": e.passed_guardrails,
                "guardrails": {
                    "ari_pass": e.ari_score <= 8,
                    "lix_pass": e.lix_score <= 40,
                    "sentence_len_pass": e.avg_sentence_len <= 15,
                    "meaning_pass": e.meaning_cosine >= 0.70,
                }
            })
        
        return aggregate_feedback(eval_dicts)
    
    def generate_tickets(self):
        """Generate development tickets from feedback."""
        summary = self.generate_feedback()
        eval_ids = [e.id for e in self.evaluations]
        self.tickets = generate_tickets_from_feedback(summary, eval_ids)
        print(f"Generated {len(self.tickets)} development tickets")
    
    def save_results(self):
        """Save all results to JSON files."""
        # Save inputs
        with open(f"{self.run_dir}/inputs.json", 'w', encoding='utf-8') as f:
            json.dump([asdict(i) for i in self.inputs], f, indent=2, ensure_ascii=False)
        
        # Save outputs
        with open(f"{self.run_dir}/outputs.json", 'w', encoding='utf-8') as f:
            json.dump([asdict(o) for o in self.outputs], f, indent=2, ensure_ascii=False)
        
        # Save evaluations
        with open(f"{self.run_dir}/evaluations.json", 'w', encoding='utf-8') as f:
            json.dump([asdict(e) for e in self.evaluations], f, indent=2, ensure_ascii=False)
        
        # Save tickets
        save_tickets_to_json(self.tickets, f"{self.run_dir}/tickets.json")
        
        # Save feedback report
        summary = self.generate_feedback()
        report = generate_feedback_report(summary)
        with open(f"{self.run_dir}/report.md", 'w', encoding='utf-8') as f:
            f.write(report)
        
        print(f"\n📁 All results saved to: {self.run_dir}")
    
    def run_full_pipeline(self):
        """Run the complete feedback loop."""
        self.run_evaluation()
        self.generate_tickets()
        self.save_results()
        
        # Print summary
        summary = self.generate_feedback()
        print("\n" + generate_feedback_report(summary))

print("✅ FeedbackLoopPipeline class implemented")

✅ FeedbackLoopPipeline class implemented


---

# 🧪 Demo: Running the Feedback Loop

This demonstrates the complete pipeline using the verification test examples.

In [11]:
# Demo: Create pipeline and add test data

pipeline = FeedbackLoopPipeline(MODEL_CONFIG)

# Add German test sources
german_sources = [
    SourceText(
        id="de_test_001",
        text="Der Antragsteller muss die erforderlichen Unterlagen innerhalb der gesetzlich vorgeschriebenen Frist beim zuständigen Amt einreichen.",
        language="de",
        domain="government",
        difficulty="complex"
    ),
    SourceText(
        id="de_test_002",
        text="Die Medikamenteneinnahme sollte zweimal täglich zu den Mahlzeiten erfolgen, um mögliche Magen-Darm-Beschwerden zu minimieren.",
        language="de",
        domain="medical",
        difficulty="complex"
    )
]

# Add English test sources
english_sources = [
    SourceText(
        id="en_test_001",
        text="Upon arrival at the facility, visitors are required to sign in at the front desk and present valid photo identification.",
        language="en",
        domain="general",
        difficulty="standard"
    ),
    SourceText(
        id="en_test_002",
        text="The medication should be administered twice daily with food to minimize potential gastrointestinal discomfort.",
        language="en",
        domain="medical",
        difficulty="complex"
    )
]

pipeline.add_source_texts(german_sources + english_sources)

Added 4 source texts


In [12]:
# Demo: Add simulated model outputs (using expected outputs as reference)

# Simulated outputs from llama-3.1-8b-instant
model_outputs = [
    ModelOutput(
        id="out_de_001",
        source_id="de_test_001",
        model="llama-3.1-8b-instant",
        prompt_version="v1.0",
        output_text="""Sie müssen Papiere abgeben.

* Bringen Sie die Papiere zum Amt.
* Sie haben dafür eine bestimmte Zeit.
* Diese Zeit steht im Gesetz.""",
        language="de",
        inference_time_ms=245.3
    ),
    ModelOutput(
        id="out_de_002",
        source_id="de_test_002",
        model="llama-3.1-8b-instant",
        prompt_version="v1.0",
        output_text="""Nehmen Sie diese Medizin zwei Mal am Tag.

* Nehmen Sie sie beim Essen.
* Das ist besser für Ihren Magen.""",
        language="de",
        inference_time_ms=198.7
    ),
    ModelOutput(
        id="out_en_001",
        source_id="en_test_001",
        model="llama-3.1-8b-instant",
        prompt_version="v1.0",
        output_text="""When you arrive:

* Go to the front desk.
* Sign your name.
* Show your photo ID.""",
        language="en",
        inference_time_ms=187.2
    ),
    ModelOutput(
        id="out_en_002",
        source_id="en_test_002",
        model="llama-3.1-8b-instant",
        prompt_version="v1.0",
        output_text="""Take this medicine two times every day.

* Eat food when you take it.
* This helps your stomach feel better.""",
        language="en",
        inference_time_ms=156.9
    )
]

pipeline.add_model_outputs(model_outputs)

Added 4 model outputs


In [13]:
# Demo: Run the full feedback loop pipeline

pipeline.run_full_pipeline()


Running evaluation for 4 outputs...


/Users/simonvoegely/Desktop/easysprache/klartext/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



✅ Evaluation complete: 3/4 passed guardrails
Generated 0 development tickets
✅ Saved 0 tickets to ../outputs/feedback_loop/runs/20260112_133231/tickets.json

📁 All results saved to: ../outputs/feedback_loop/runs/20260112_133231

# 📊 Feedback Loop Report

**Generated**: 2026-01-12 13:32

## Summary
- Total Evaluations: 4
- Pass Rate: 75.0%

## Metrics
| Metric | Value | Target | Status |
|--------|-------|--------|--------|
| ARI Score | 1.5 | ≤ 8 | ✅ |
| LIX Score | 15.6 | ≤ 40 | ✅ |
| Avg Sentence Len | 5.7 | ≤ 10 | ✅ |
| Meaning Similarity | 0.76 | ≥ 0.70 | ✅ |


---

# 📋 Ticket Tracking Dashboard

In [14]:
# Display generated tickets

def display_tickets(tickets: List[DevelopmentTicket]):
    """Display tickets in a formatted view."""
    if not tickets:
        print("✅ No tickets generated - all evaluations passed!")
        return
    
    print("\n" + "="*70)
    print("📋 DEVELOPMENT TICKETS")
    print("="*70)
    
    for ticket in tickets:
        priority_icon = {
            "critical": "🔴",
            "high": "🟠",
            "medium": "🟡",
            "low": "🟢"
        }
        
        print(f"\n{priority_icon.get(ticket.priority, '⚪')} [{ticket.id}] {ticket.title}")
        print(f"   Status: {ticket.status.upper()} | Type: {ticket.ticket_type}")
        print(f"   Action: {ticket.suggested_action}")
        print(f"   Acceptance Criteria:")
        for ac in ticket.acceptance_criteria:
            print(f"      - {ac}")

display_tickets(pipeline.tickets)

✅ No tickets generated - all evaluations passed!


---

# 📁 JSON File Structure

The feedback loop creates the following JSON file structure:

```
outputs/feedback_loop/
├── runs/
│   └── YYYYMMDD_HHMMSS/
│       ├── inputs.json        # Source texts
│       ├── outputs.json       # Model outputs
│       ├── evaluations.json   # Evaluation results
│       ├── tickets.json       # Generated tickets
│       └── report.md          # Human-readable report
└── tickets/
    └── backlog.json           # Aggregated ticket backlog
```

In [15]:
# Example: View JSON output structure

print("\n📄 Sample Evaluation JSON:")
print(json.dumps(asdict(pipeline.evaluations[0]), indent=2, ensure_ascii=False))


📄 Sample Evaluation JSON:
{
  "id": "eval_out_de_001",
  "output_id": "out_de_001",
  "source_id": "de_test_001",
  "ari_score": 3.85,
  "lix_score": 29.06,
  "avg_sentence_len": 5.2,
  "max_sentence_len": 6,
  "pct_long_sentences": 0.0,
  "meaning_cosine": 0.655,
  "rule_violations": [],
  "rule_score": 1.0,
  "passed_guardrails": false,
  "overall_score": 0.768,
  "timestamp": "2026-01-12T13:32:42.754238"
}


---

# ✅ System Verification Checklist

## Completed Features

| Feature | Status | Notes |
|---------|--------|-------|
| JSON schema for source texts | ✅ | `SourceText` dataclass |
| JSON schema for model outputs | ✅ | `ModelOutput` dataclass |
| JSON schema for evaluations | ✅ | `EvaluationResult` dataclass |
| JSON schema for tickets | ✅ | `DevelopmentTicket` dataclass |
| ARI scoring | ✅ | Works for EN + DE |
| LIX scoring | ✅ | Standard readability metric |
| Sentence metrics | ✅ | avg/max/pct_long |
| Meaning similarity | ✅ | sentence-transformers + TF-IDF fallback |
| Rule violation detection | ✅ | Passive, negations, length, intro text, XML |
| Verification test suite | ✅ | Based on prompt template examples |
| Feedback aggregation | ✅ | Summarizes patterns and recommendations |
| Ticket generation | ✅ | Auto-generates from findings |
| Complete pipeline | ✅ | `FeedbackLoopPipeline` class |
| Model configuration | ✅ | `llama-3.1-8b-instant` via Groq |

---

# 🔄 Next Steps

1. **Connect to live model**: Add Groq API integration to generate real outputs
2. **Expand test suite**: Add more verification tests from production use cases
3. **Historical tracking**: Compare results across multiple runs
4. **Dashboard visualization**: Create charts for metric trends